## read data from files netatmo

In [1]:
import numpy as np
#import tensorflow as tf
#from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters

from sklearn.model_selection import cross_val_score

%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

In [2]:
# supprimer d'abord les deux lignes dans le fichier d'export Neatatmo
index_col="date"
data=pd.read_csv('X_selected.csv', sep=",", encoding = "ISO-8859-1", index_col=index_col)



In [3]:
data

,co2_100,temperature_100,co2_101,temperature_101,co2_102,temperature_105,co2_106,light_106,co2_107,co2_108,sound_108,humidity_110,weather_out,weather_hum,weather_bar_,windows
date,,,,,,,,,,,,,,,,
2022-03-15 00:00:00,509.333333,25.893333,443.666667,25.450000,473.000000,25.730000,514.666667,0.0,572.666667,577.50,5.500000,25.640000,11.600000,66.000000,1023.10,1
2022-03-15 00:05:00,504.250000,25.895000,439.666667,25.453333,468.333333,25.737500,514.750000,0.0,569.500000,577.00,5.666667,25.747500,11.616667,66.333333,1023.05,1
2022-03-15 00:10:00,500.500000,25.900000,437.000000,25.460000,466.666667,25.726667,515.000000,0.0,568.000000,575.25,5.000000,25.775000,11.633333,66.666667,1023.00,1
2022-03-15 00:15:00,499.666667,25.890000,439.666667,25.450000,464.777778,25.713333,514.666667,0.0,565.000000,572.00,6.000000,25.872500,11.650000,67.000000,1022.95,1
2022-03-15 00:20:00,498.833333,25.880000,442.333333,25.440000,462.888889,25.702500,512.750000,0.0,568.250000,567.75,5.000000,25.766667,11.666667,67.333333,1022.90,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-30 23:35:00,405.000000,24.535000,410.250000,24.672500,424.500000,25.118000,428.000000,0.0,419.750000,390.20,9.200000,29.114000,14.100000,54.000000,1013.20,1
2022-05-30 23:40:00,405.800000,24.524000,409.500000,24.667500,424.750000,25.097500,425.000000,0.0,422.400000,390.00,9.000000,29.100000,14.100000,54.000000,1013.20,1
2022-05-30 23:45:00,403.500000,24.527500,412.600000,24.674000,426.000000,25.097500,420.500000,0.0,422.250000,390.80,9.000000,29.074000,14.000000,55.000000,1013.20,1


## Missing data generation

In [13]:

#!pip install wget 
import wget
#wget.download('https://raw.githubusercontent.com/BorisMuzellec/MissingDataOT/master/utils.py')

import numpy as np
import pandas as pd
from utils import *
import torch
import seaborn as sns

In [14]:
# Fix the seed ------------------------------------------------------
np.random.seed(2)

In [15]:
# Function produce_NA for generating missing values ------------------------------------------------------

def produce_NA(X, p_miss, mecha="MCAR", opt=None, p_obs=None, q=None):
    """
    Generate missing values for specifics missing-data mechanism and proportion of missing values. 
    
    Parameters
    ----------
    X : torch.DoubleTensor or np.ndarray, shape (n, d)
        Data for which missing values will be simulated.
        If a numpy array is provided, it will be converted to a pytorch tensor.
    p_miss : float
        Proportion of missing values to generate for variables which will have missing values.
    mecha : str, 
            Indicates the missing-data mechanism to be used. "MCAR" by default, "MAR", "MNAR" or "MNARsmask"
    opt: str, 
         For mecha = "MNAR", it indicates how the missing-data mechanism is generated: using a logistic regression ("logistic"), quantile censorship ("quantile") or logistic regression for generating a self-masked MNAR mechanism ("selfmasked").
    p_obs : float
            If mecha = "MAR", or mecha = "MNAR" with opt = "logistic" or "quanti", proportion of variables with *no* missing values that will be used for the logistic masking model.
    q : float
        If mecha = "MNAR" and opt = "quanti", quantile level at which the cuts should occur.
    
    Returns
    ----------
    A dictionnary containing:
    'X_init': the initial data matrix.
    'X_incomp': the data with the generated missing values.
    'mask': a matrix indexing the generated missing values.s
    """
    
    to_torch = torch.is_tensor(X) ## output a pytorch tensor, or a numpy array
    if not to_torch:
        X = X.astype(np.float32)
        X = torch.from_numpy(X)
    
    if mecha == "MAR":
        mask = MAR_mask(X, p_miss, p_obs).double()
    elif mecha == "MNAR" and opt == "logistic":
        mask = MNAR_mask_logistic(X, p_miss, p_obs).double()
    elif mecha == "MNAR" and opt == "quantile":
        mask = MNAR_mask_quantiles(X, p_miss, q, 1-p_obs).double()
    elif mecha == "MNAR" and opt == "selfmasked":
        mask = MNAR_self_mask_logistic(X, p_miss).double()
    else:
        mask = (torch.rand(X.shape) < p_miss).double()
    
    X_nas = X.clone()
    X_nas[mask.bool()] = np.nan
    
    return {'X_init': X.double(), 'X_incomp': X_nas.double(), 'mask': mask}

In [16]:
X = data.copy()
# X = X.values
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# datascaler=scaler.fit(X)
# X= datascaler.transform(X)

In [17]:
import pandas as pd
df=X.copy()
df.index=data.index

In [18]:
df.head()

,co2_100,temperature_100,co2_101,temperature_101,co2_102,temperature_105,co2_106,light_106,co2_107,co2_108,sound_108,humidity_110,weather_out,weather_hum,weather_bar_,windows
date,,,,,,,,,,,,,,,,
2022-03-15 00:00:00,509.333333,25.893333,443.666667,25.450000,473.000000,25.730000,514.666667,0.0,572.666667,577.50,5.500000,25.640000,11.600000,66.000000,1023.10,1
2022-03-15 00:05:00,504.250000,25.895000,439.666667,25.453333,468.333333,25.737500,514.750000,0.0,569.500000,577.00,5.666667,25.747500,11.616667,66.333333,1023.05,1
2022-03-15 00:10:00,500.500000,25.900000,437.000000,25.460000,466.666667,25.726667,515.000000,0.0,568.000000,575.25,5.000000,25.775000,11.633333,66.666667,1023.00,1
2022-03-15 00:15:00,499.666667,25.890000,439.666667,25.450000,464.777778,25.713333,514.666667,0.0,565.000000,572.00,6.000000,25.872500,11.650000,67.000000,1022.95,1
2022-03-15 00:20:00,498.833333,25.880000,442.333333,25.440000,462.888889,25.702500,512.750000,0.0,568.250000,567.75,5.000000,25.766667,11.666667,67.333333,1022.90,1


## MCAR Generation

In [64]:
# Minimal example for generating missing data ------------------------
p='70.csv'
X_miss_mcar = produce_NA(X.values, p_miss=0.70, mecha="MCAR")
X_mcar = X_miss_mcar['X_incomp']
R_mcar = X_miss_mcar['mask']

print("Percentage of newly generated missing values: ", (R_mcar.sum()).numpy()/np.prod(R_mcar.size())*100, " %")

df = X_mcar
df = df.numpy()
mask=R_mcar
mask = mask.numpy()

df = pd.DataFrame(data=df, columns=X.columns)
mask = pd.DataFrame(data=mask, columns=X.columns)

df['date']= data.index
df.set_index('date',inplace = True)
mask['date']= data.index
mask.set_index('date',inplace = True)
df.to_csv('MCAR'+p)
mask.to_csv('mask'+p)

Percentage of newly generated missing values:  70.0549016955267  %


In [122]:
#df.to_csv('MCAR25.csv')
# supprimer d'abord les deux lignes dans le fichier d'export Neatatmo
index_col="date"
df=pd.read_csv('MCAR40.csv', sep=",", encoding = "ISO-8859-1", index_col=index_col)
mask=pd.read_csv('mask40.csv', sep=",", encoding = "ISO-8859-1", index_col=index_col)
mask=mask.values


In [68]:
# Sample mar data 
p='60.csv'
X_miss_mar = produce_NA(X.values, p_miss=0.65, mecha="MAR", p_obs=0.1)

X_mar = X_miss_mar['X_incomp']
R_mar = X_miss_mar['mask']

print("Percentage of generated missing values: ", (R_mar.sum()).numpy()/np.prod(R_mar.size())*100, " %")

# ax = sns.heatmap(X_mar, mask=R_mar.numpy()==1, linewidths=0.02, linecolor='black')

df = X_mar
df = df.numpy()
mask_mar = R_mar
mask_mar = mask_mar.numpy()

df = pd.DataFrame(data=df, columns=X.columns)
mask = pd.DataFrame(data=mask, columns=X.columns)

df['date']= data.index
df.set_index('date',inplace = True)
mask['date']= data.index
mask.set_index('date',inplace = True)
df.to_csv('MAR'+p)
mask.to_csv('maskMAR'+p)


Percentage of generated missing values:  60.831811417748916  %


In [86]:
import numpy as np
j=0
for i in X_mar[:,0].numpy():
    if np.isnan(i) :
        j=j+1
       
    else:
        print(j)
        j=0

0
0
14
0
0
0
5
0
0
2
0
0
2
1
1
0
4
6
3
1
1
3
0
1
0
0
8
4
1
1
0
0
2
6
19
34
1
3
6
13
10
0
28
1
3
2
4
1
0
2
8
10
1
4
3
0
4
0
2
4
5
0
0
1
3
2
6
0
2
4
0
0
2
3
2
3
0
1
0
0
4
0
2
0
1
1
7
0
0
3
1
0
3
1
5
0
2
1
4
0
10
40
5
2
2
4
3
3
6
5
4
11
5
9
9
0
15
0
5
0
2
3
10
0
0
0
2
1
0
6
2
0
5
2
0
0
2
0
1
1
2
4
2
2
1
3
3
2
2
6
1
1
1
0
3
5
1
0
0
0
60
1
5
3
2
1
1
1
3
6
5
2
13
2
1
1
10
4
0
1
0
1
1
1
0
2
3
2
0
0
0
2
8
0
1
1
3
2
0
0
0
6
2
6
0
0
2
4
0
1
0
4
2
2
0
5
1
1
4
3
0
1
0
0
0
0
3
2
1
2
1
0
2
1
1
1
2
62
5
0
5
21
24
0
0
2
0
0
1
0
0
0
1
5
0
5
2
1
0
1
1
0
0
3
0
0
3
0
1
5
1
1
0
1
0
0
5
0
0
0
1
0
1
0
0
1
0
0
0
0
3
2
0
1
2
1
0
3
0
4
0
1
3
0
0
0
0
2
1
0
1
3
0
3
2
0
2
0
2
2
1
2
1
0
0
0
1
1
0
3
1
3
1
5
0
3
1
2
1
1
0
0
0
1
0
1
0
1
0
3
0
0
8
3
2
0
1
0
5
1
0
1
0
0
2
2
1
2
1
0
0
6
3
0
0
1
3
2
1
3
0
0
5
0
0
1
1
0
0
4
4
1
1
0
1
1
3
1
2
3
3
1
3
0
1
0
10
1
9
1
3
7
0
2
0
1
0
1
0
1
0
1
1
0
0
3
2
0
3
1
1
8
1
1
2
1
1
0
2
0
1
2
1
2
2
1
0
1
1
0
0
0
1
3
3
0
1
0
1
1
3
3
4
1
0
0
1
0
3
1
0
2
0
0
11
6
0
3
0
1
1
0
6
7
2
2
2
0
0
5


In [81]:
X_mar[:,0].numpy().isnan()

AttributeError: 'numpy.ndarray' object has no attribute 'isnan'

In [ ]:
# df['date']= data.index
# df.set_index('date',inplace = True)
# mask_mar['date'] = data.index
# mask_mar.set_index('date',inplace = True)
# df.to_csv('MAR'+p)
# mask_mar.to_csv('mask_mar'+p)

In [38]:
# Minimal example for generating missing data ------------------------
p='5.csv'

In [ ]:
pourcentage=5
p=